In [3]:
import os
import json
import jsonlines

from itertools import combinations
from collections import defaultdict, Counter
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix
from scipy.spatial import distance

In [4]:
def read_memmap(filepath):
    with open(filepath.replace(".dat", ".conf"), "r") as fin_config:
        memmap_configs = json.load(fin_config)
        return np.memmap(filepath, mode="r", shape=tuple(memmap_configs["shape"]), dtype=memmap_configs["dtype"])

In [5]:
NUM_FORMATS = 8

In [6]:
result_dir = "../../../results"
input_dir = "../../../preprocessed_datasets"

In [7]:
model_name = "Llama-3.1-8B-Instruct"
dataset_name = "CommonsenseQA"
prompting_strategy = "zero-shot"

output_dir = f"{result_dir}/{dataset_name}/{model_name}"

predictions_path = os.path.join(output_dir, f"{prompting_strategy}_predictions_validation.jsonl")
try:
    with jsonlines.open(predictions_path) as fin:
        id_predictions_map = {}
        for example in fin.iter():
            id_predictions_map[example["id"]] = example["predictions"]
except:
    pass

labels = []
for preds in id_predictions_map.values():
    pairwise_labels = []
    for i, j in list(combinations(range(8), 2)):
        label = int(preds[str(i)] == preds[str(j)])
        pairwise_labels.append(label)
    labels.append(pairwise_labels)
labels = np.array(labels)


In [8]:
def expand_pairwise_embeddings(hidden_states):
    num_samples = hidden_states.shape[0]

    pairwise_embeddings = []

    for s in range(num_samples):
        sample_pairs = []
        for i, j in list(combinations(range(8), 2)):
            emb_i = hidden_states[s, i]
            emb_j = hidden_states[s, j]
            concat = np.concatenate([emb_i, emb_j], axis=-1)
            sample_pairs.append(concat)
        sample_pairs = np.stack(sample_pairs, axis=0)
        pairwise_embeddings.append(sample_pairs)

    return np.stack(pairwise_embeddings, axis=0)

In [9]:
layer_wise_path = os.path.join(output_dir, f"{prompting_strategy}_layer_wise_hidden_states_validation.dat")
head_wise_path = os.path.join(output_dir, f"{prompting_strategy}_head_wise_hidden_states_validation.dat")

layer_wise_hidden_states = read_memmap(layer_wise_path)
head_wise_hidden_states = read_memmap(head_wise_path)

pairwise_layer_wise_hidden_states = expand_pairwise_embeddings(layer_wise_hidden_states)
pairwise_head_wise_hidden_states = expand_pairwise_embeddings(head_wise_hidden_states)

In [ ]:
num_samples, num_formats, num_layers, hidden_size = layer_wise_hidden_states.shape
num_samples, num_formats, num_layers, num_heads, hidden_size2 = head_wise_hidden_states.shape

layer_accuracy_scores = []
for layer_idx in range(num_layers):
    # if layer_idx not in [31]:
    #     continue

    # Step 1: Prepare input
    X = pairwise_layer_wise_hidden_states[:,:,layer_idx,:]
    X = X.reshape(-1, hidden_size*2)
    Y = labels.reshape(-1)

    # Step 2: 80:20 split for validation to select top-K components
    X_fold1, X_fold2, y_fold1, y_fold2 = train_test_split(X, Y, test_size=0.2, random_state=0)

    for i in range(2):
        # Step 3: Standardization
        scaler = StandardScaler()
        if i == 0:
            X_train = scaler.fit_transform(X_fold1)
            X_test = scaler.transform(X_fold2)
            y_train, y_test = y_fold1, y_fold2
        else:
            break
            X_train = scaler.fit_transform(X_fold2)
            X_test = scaler.transform(X_fold1)
            y_train, y_test = y_fold2, y_fold1

        # Step 4: Train a linear probing model
        clf = LogisticRegression(solver="lbfgs", max_iter=1000)
        clf.fit(X_train, y_train)

        # Step 5: Evaluate the model
        y_pred = clf.predict(X_test)
        accuracy = balanced_accuracy_score(y_test, y_pred)
        layer_accuracy_scores.append(accuracy)
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
    print(layer_accuracy_scores)
    print("*"*20)

In [ ]:
print(sorted(layer_accuracy_scores, reverse=True)[:10])
print()
print(sorted(layer_accuracy_scores, reverse=False)[:10])

In [ ]:
num_samples, num_formats, num_layers, hidden_size = layer_wise_hidden_states.shape
num_samples, num_formats, num_layers, num_heads, hidden_size2 = head_wise_hidden_states.shape

head_accuracy_scores = defaultdict(list)
for layer_idx in range(num_layers):
    # if layer_idx not in [31]:
    #     continue
    for head_idx in range(num_heads):
        # Step 1: Prepare input
        X = pairwise_head_wise_hidden_states[:,:,layer_idx,head_idx,:]
        X = X.reshape(-1, hidden_size2*2)
        Y = labels.reshape(-1)

        # Step 2: 80:20 split for validation to select top-K components
        X_fold1, X_fold2, y_fold1, y_fold2 = train_test_split(X, Y, test_size=0.2, random_state=0)

        for i in range(2):
            # Step 3: Standardization
            scaler = StandardScaler()
            if i == 0:
                X_train = scaler.fit_transform(X_fold1)
                X_test = scaler.transform(X_fold2)
                y_train, y_test = y_fold1, y_fold2
            else:
                break
                X_train = scaler.fit_transform(X_fold2)
                X_test = scaler.transform(X_fold1)
                y_train, y_test = y_fold2, y_fold1

            # Step 4: Train a linear probing model
            clf = LogisticRegression(solver="lbfgs", max_iter=1000)
            clf.fit(X_train, y_train)

            # Step 5: Evaluate the model
            y_pred = clf.predict(X_test)
            accuracy = balanced_accuracy_score(y_test, y_pred)
            head_accuracy_scores[layer_idx].append(accuracy)
            # print(classification_report(y_test, y_pred))
            # print(confusion_matrix(y_test, y_pred))
    print(head_accuracy_scores)
    print("*"*20)

In [ ]:
print(sorted(np.array(list(head_accuracy_scores.values())).flatten().tolist(), reverse=True)[:10])
print()
print(sorted(np.array(list(head_accuracy_scores.values())).flatten().tolist(), reverse=False)[:10])

In [ ]:
def get_topk_heads(scores, k=10):
    num_layers, num_heads = scores.shape
    flat_scores = scores.flatten()
    topk_indices = np.argsort(flat_scores)[-k:][::-1]

    topk_info = []
    for idx in topk_indices:
        layer = idx // num_heads
        head = idx % num_heads
        score = scores[layer, head]
        topk_info.append(((int(layer), int(head)), float(score)))

    return topk_info

aaa = np.array(list(head_accuracy_scores.values()))
print(get_topk_heads(aaa, 25))

top_k_heads_path = os.path.join(output_dir, f"{prompting_strategy}_activation_steering_top_k_heads.json")
with open(top_k_heads_path, "w") as fout:
    json.dump(get_topk_heads(aaa, 25), fout)

In [ ]:
def get_topk_layers(layer_scores, k=10):
    topk_indices = np.argsort(layer_scores)[-k:][::-1]
    topk_info = [(idx, layer_scores[idx]) for idx in topk_indices]
    return topk_info

get_topk_layers(layer_accuracy_scores, 10)

## Computing Steering Direction

In [14]:
z = []
for id, predictions in id_predictions_map.items():
    if "majority_voting" in predictions:
        predictions.pop("majority_voting")
    zz = []
    for ii in range(NUM_FORMATS):
        zzz = list(predictions.values()).count(predictions[str(ii)])
        zzz = ((zzz>4)*1.0)
        zz.append(zzz)
    z.append(zz)
majority_minority_array = np.array(z)

In [59]:
# def extract_embeddings_by_class(embeddings, majority_minority_array):
#     N, F, L, H, D = embeddings.shape

#     flat_embeddings = embeddings.reshape(N * F, L, H, D)
#     flat_labels = majority_minority_array.reshape(N * F)

#     positive_embeddings = flat_embeddings[flat_labels > 0.5]
#     negative_embeddings = flat_embeddings[flat_labels < 0.5]

#     return positive_embeddings, negative_embeddings

def extract_embeddings_by_class(embeddings, majority_minority_array):
    N, F, L, H, D = embeddings.shape

    selection_idx = np.logical_and(majority_minority_array.mean(1) < 1, majority_minority_array.mean(1) > 0)
    selected_embeddings = embeddings[selection_idx]
    selected_labels = majority_minority_array[selection_idx]

    # (1) without balancing
    # selected_N = int(selection_idx.sum())
    # flat_embeddings = selected_embeddings.reshape(selected_N * F, L, H, D)
    # flat_labels = selected_labels.reshape(selected_N * F)
    # positive_embeddings = flat_embeddings[flat_labels > 0.5]
    # negative_embeddings = flat_embeddings[flat_labels < 0.5]

    # (2) balancing
    pos_weight = selected_labels/((selected_labels.sum(1))[...,np.newaxis])
    neg_weight = (1-selected_labels)/(((1-selected_labels).sum(1))[...,np.newaxis])
    positive_embeddings = (selected_embeddings*pos_weight[...,np.newaxis,np.newaxis,np.newaxis]).sum(1)
    negative_embeddings = (selected_embeddings*neg_weight[...,np.newaxis,np.newaxis,np.newaxis]).sum(1)

    return positive_embeddings, negative_embeddings

In [60]:
pos, neg = extract_embeddings_by_class(head_wise_hidden_states, majority_minority_array)
steering_direction = pos.mean(0) - neg.mean(0)

steering_direction_path = os.path.join(output_dir, f"{prompting_strategy}_activation_steering_direction.npy")
np.save(steering_direction_path, steering_direction)